#Eliza Chatbot

This code implements a chatbot system that engages in conversations based on predefined patterns and responses. It involves filtering user input to handle contractions and replacements, identifying keywords in the input to trigger appropriate responses, and logging conversation history including potential keywords for further script refinement. The chatbot continuously interacts with users, responding to their inputs until the user decides to exit the conversation.

In [ ]:
# @title Import Libraries
import re
import random
import logging
from datetime import datetime


In [ ]:
# @title Functions
def filter_contractions(prompt):
    contractions_dict = {
        r"(\w*?)(won't|'ll)\b": lambda match: (match.group(1) + " will" if match.group(1) else "will not"),
        r"(\w*?)(can't|n't)\b": lambda match: (match.group(1) if match.group(1) else "can") + " not",
        r"(\w+)'s\b": lambda match: match.group(1) + " is",
        r"(\w+)'ve\b": lambda match: match.group(1) + " have",
        r"(\w*?)'re\b": lambda match: match.group(1) + " are",
        r".*\bi'm\b": "i am"
    }

    # Function to find matching contractions or possessive forms and replace them with expanded forms
    for pattern, replacement in contractions_dict.items():
        prompt = re.sub(pattern, replacement, prompt, flags=re.IGNORECASE)

    return prompt

def filter_replacements(prompt):
    replacements = {
        r'\b(am)\b': 'are',
        r'\b(was)\b': 'were',
        r'\b(i)\b': 'you',
        r'\b(my)\b': 'your',
        r'\b(are)\b': 'am',
        r'\b(your)\b': 'my',
        r'\b(yours)\b': 'mine',
        r'\b(mine)\b': 'yours',
        r'\b(you)\b': 'I',
        r'\b(me)\b': 'you',
    }

    # Create a regex pattern by joining all replacement patterns
    pattern = re.compile('|'.join(replacements.keys()), re.IGNORECASE)

    # Replacement function to replace matched patterns
    def replace(match):
      key = match.group(0)
      for pattern, value in replacements.items():
          if re.match(pattern, key, re.IGNORECASE):
              return value
      return key

    # Apply replacements using the replacement function
    simplified_prompt = pattern.sub(replace, prompt)

    return simplified_prompt

def filter_prompt(prompt):
  prompt = filter_contractions(prompt)
  prompt = filter_replacements(prompt)

  return prompt

def get_keystack(message, script):
    keystack = []
    sentences = re.split(r'[.,!?]', message)
    for keyword in script.patterns.keys():
        pattern = re.compile(rf'\b(?:{keyword})\b', re.IGNORECASE)
        for sentence in sentences:
            if pattern.search(sentence):
                keystack.append((keyword, sentence.strip()))  # Remove leading/trailing whitespace
    return keystack

def respond(message):
    keystack = get_keystack(message, script)
    logging.info(f"Keystack: {keystack}")

    while keystack:  # Continue until the stack is empty
        keyword, sentence = keystack.pop(0)  # Pop the top keyword and sentence from the stack

        logging.info(f"Searching for: {keyword}")

        script_dict = script.patterns[keyword]
        for pattern, responses in script_dict.items():
            match = re.match(pattern, sentence, re.IGNORECASE)
            if match:
                response = responses.pop(0)
                responses.append(response)
                if "NEWKEY" in response:
                    # Break out of the current keyword loop
                    break
                elif "GOTO" in response:
                    # Go to the new keyword
                    new_keyword = response.split()[1]
                    keystack.insert(0, (new_keyword, sentence))
                    logging.info(f"Changed Keyword: {new_keyword}")
                    break
                elif "MEMORY" in response:
                    # Add captured response to memory
                    new_memory = match.group(1)  # The first item that got captured
                    script.memory["items"].append(new_memory)
                    logging.info(f"New Memory: {new_memory}")
                    break

                # Empty the keystack after responding
                keystack.clear()
                return response.format(*[re.sub(r'\W+$', '', g) for g in match.groups()])

    # If no keywords found, log the sentence and empty the keystack
    possible_keywords.append(f"{message}\n")
    keystack.clear()

    # There's a 40% chance to return a memory prompt if there are any
    if script.memory["items"] and random.random() < 0.4:
        return random.choice(script.memory["prompts"]) + " " + random.choice(script.memory["items"])
    else:
        return random.choice(script.automated["CONFUSED"])


In [ ]:
# @title Main { vertical-output: true, display-mode: "form" }
# @markdown Run this cell to start the conversation
script_choice = "Rubber Duck" # @param ["Rubber Duck", "DOCTOR"]

# Add the script choices as keys
scripts = {
    "Rubber Duck" : DuckScript(),
    "DOCTOR": DoctorScript()
}

script = scripts[script_choice]

# List to store conversation history
possible_keywords = []

try:
    # Set up logging
    logging.basicConfig(level=logging.INFO)

    # List to store conversation history
    conversation_history = []

    print("This is a chatbot. Type 'quit' to exit the session.")
    # Choose Greeting
    greeting_msg = random.choice(script.automated["HELLO"])
    print(f"{script.name}: {greeting_msg}")

    while True:
        user_input = input("You: ")
        conversation_history.append(f"You: {user_input}\n")
        if user_input.upper() == "QUIT":
            # Choose Goodbye
            goodbye_msg = random.choice(script.automated["GOODBYE"])
            print(f"{script.name}: {goodbye_msg}")
            break
        else:
            user_input = filter_prompt(user_input)
            response = respond(user_input)
            conversation_history.append(f"{script.name}: {response}\n")
            print(f"{script.name}:", response)

finally:
    # Get current date
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Write conversation history to a .txt file
    with open("chatlog.txt", "a") as file:
        file.write(f"\nDate: {current_date}\n")
        file.writelines(conversation_history)

    # Write possible keywords to a .txt file
    with open("possible_keywords.txt", "a") as file:
        file.writelines(possible_keywords)


This is a chatbot. Type 'quit' to exit the session.
🐤: Quack... I mean hello! Care to debug some code?
You: yes
🐤: I like your attitude!
You: I'm printing 0-9 instead of 1-10
🐤: Why?
You: I don't know! I have a loop that iterates 10 times but the prints are wrong
🐤: Why?
You: this is my code for i in range(10): print(i)
🐤: I already told you, you have to explain your code verbally.
You: well, it has a range of 10 so it starts counting from 1
🐤: Why?
You: wait no! it starts counting from 0
🐤: Why not?
You: thanks!
🐤: No problem.
You: quit
🐤: See ya!


In [ ]:
# @title Duck Script
class DuckScript:
    def __init__(self):
        self.name = "🐤"

        self.memory = {
            "prompts": [
                "Let's discuss further why your",
                "Earlier you talked about your",
                "But your"
            ],
            "items": []
        }

        self.tags = {
            "FAMILY": ["mother|mom", "father|dad", "wife", "children", "sister", "brother"],
            "GREETINGS": ["hi|hii|hiii", "hello|helo", "hey|heyy|heyyy|heyo", "aloha"],
            "PROG_LANGUAGE": ["java", "c", "c++", "python", "javascript"],
            "CODE": ["if|else|elif", "switch", "def", "try|catch", "{|}", "while|for", "true|false", "print"],
        }

        self.automated = {
            "HELLO": [
                "Hello there! I'm your friendly rubber duck debugger.",
                "Quack... I mean hello! Care to debug some code?",
                "Hello! What's quack-a-lackin'?"
            ],
            "GOODBYE": [
                "Goodbye!",
                "See ya!"
            ],
            "CONFUSED": [
                "Please tell me more...",
                "I'm not sure I understand, can you rephrase that?",
                "Wanna talk about something else?",
                "I see...",
                "Go on...",
                "Why?"
            ]
        }

        self.patterns = {
            r"DEQUACK": {
                r'\bDEQUACK REFRESH\b': [
                    "RESPONSE 1",
                    "RESPONSE 2"
                ],
                r'\bDEQUACK MEMORY\b (.*)': [
                    "MEMORY {0}"
                ],
                rf'\bDEQUACK TAG {"|".join(self.tags["PROG_LANGUAGE"])}\b': [
                    "{0} IS A PROGRAMMING LANGUAGE"
                ]
            },
            rf'{"|".join(self.tags["GREETINGS"])}': {
                r'.*': [
                    "What seems to be the problem?",
                    "You already said hello."
                ]
            },
            r"YOUR": {
                r'\byour (.*)': [
                    "MEMORY {0}"
                ]
            },
            rf'{"|".join(self.tags["CODE"])}': {
                r".*\w\(.*\)|.*=.*|.*[:|;]$|.*[\(|{|\[].*[\]|\}|\)]": [
                    "I'm a duck, I can't understand code. What does this do?",
                    "I already told you, you have to explain your code verbally."
                ],
                r".*\bif\b.*": [
                    "GOTO IF"
                ],
                r".*": [
                    "NEWKEY"
                ]
            },
            r'YES|YEAH|MHM': {
                r'.*': [
                    "I like your attitude!"
                ]
            },
            r'NO|NAH': {
                r".*": [
                    "Why not?",
                    "Why are you being negative?"
                ]
            },
            r'ERROR|ERRORS': {
                r".*": [
                    "What do you think this error means?",
                    "Where could you find information about this error?"
                ]
            },
            r'PROBLEM|PROBLEMS': {
                r".*": [
                    "Is it a big problem?",
                    "Why is there a problem?",
                    "Can you explain the problem to me?"
                ]
            },
            r"WORK|WORKS": {
                r'(.*) \bdoes not work.*': [
                    "Why do you think {0} doesn't work?",
                    "Have you tried turning {0} off and on again?",
                    "Explain {0} a bit more."
                ]
            },
            r"SHOULD": {
                r'(\w+)\s+\bshould not\b.*': [ #Captures exactly one word before should
                    "What should {0} be doing?"
                ],
                r'(\w+)\s+\bshould\b (.*)': [
                    "Why should {0} {1}?"
                ]
            },
            r"HAVE": {
                r'.*\byou have\b (.*)': [
                    "Why do you have {0}?",
                    "How long have you had {0}?",
                    "Would you be better without {0}?"
                ]
            },
            r"THANK|THANKS": {
                r".*": [
                    "You're welcome!",
                    "No problem."
                ]
            },
            r"KNOW": {
                r'.*\byou do not know\b (.*)': [
                    "What do you know?",
                    "Give me the first reason that comes to mind."
                ]
            },
            r"THERE": {
                r".*\bthere (is|are)\b (.*)": [
                    "Why {0} there {1}?"
                ]
            },
            r"IF": {
                r'.*\bif\b (.*)': [
                    "Do you think it's likely that {0}?",
                    "Do you wish that {0}?",
                    "What do you think about {0}?",
                    "Really, if {0}?"
                ]
            },
            r'MAYBE|PERHAPS': {
                r'.*': [
                    "You don't seem quite certain",
                    "Why the uncertain tone?",
                    "Don't you know?"
                ]
            },
            r"WHAT|HOW|WHEN": {
                r'.*': [
                    "Why do you ask?",
                    "What do you think?",
                    "What comes to your mind when you ask that?",
                    "Have you asked anyone else?"
                ]
            },
            r"WANT": {
                r'\byou do not want\b (.*)': [
                    "What do you want instead?",
                    "Why don't you want {0}"
                ],
                r'\byou want (.*)': [
                    "How would you feel if you got {0}?",
                    "Why do you want {0}?"
                ]
            },
            r'ARE': {
                r'.*\byou are\b (.*)': [
                    "Why do you think you're {0}?",
                    "Do you like being {0}?"
                ]
            }
        }

In [ ]:
# @title DOCTOR script
class DoctorScript:
    def __init__(self):
      self.name = "ELIZA"

      self.memory= {
          # Prompts
          "prompts": [
          "LET'S DISCUSS FURTHER WHY YOUR",
          "EARLIER YOU SAID YOUR",
          "BUT YOUR",
          "DOES THAT HAVE ANYTHING TO DO WITH THE FACT THAT YOUR"],
          # Remembered items will be appended here
          "items": []
      }

      self.tags = {
          "FAMILY" : ["mother|mom", "father|dad", "wife", "children", "sister", "brother"],
          "BELIEF": ["feel", "think", "believe", "wish"]
      }


      self.automated = {
          "HELLO": [
              "HELLO, I'M ELIZA AND I'M HERE TO HELP"
          ],
          "GOODBYE": [
              "GOODBYE",
              "FAREWELL"
          ],
          "CONFUSED": [
              "I'M NOT SURE I UNDERSTAND YOU FULLY.",
              "PLEASE GO ON",
              "WHAT DOES THAT SUGGEST TO YOU",
              "DO YOU FEEL STRONGLY ABOUT DISCUSSING SUCH THINGS."
          ]
      }




      self.patterns = {
          r"HELLO": {
              r'.*': [
                  "HOW DO YOU DO? PLEASE STATE YOUR PROBLEM"
              ]
          },

          r'DEUTSCH|FRANCAIS|ITALIANO|ESPANOL': {
              r'.*': [
                  "I AM SORRY, I SPEAK ONLY ENGLISH"
              ]
          },

          r"SORRY": {
              r'.*\bSORRY\b.*': [
                  "PLEASE DON'T APOLOGIZE",
                  "APOLOGIES ARE NOT NECESSARY",
                  "WHAT FEELINGS DO YOU HAVE WHEN YOU APOLOGIZE?",
                  "I'VE TOLD YOU THAT APOLOGIES ARE NOT REQUIRED"
              ]
          },

          r"REMEMBER": {
              r'.*\bYOU REMEMBER\b (.*)': [
                  "DO YOU OFTEN THINK OF {0}",
                  "DOES THINKING OF {0} BRING ANYTHING ELSE TO MIND",
                  "WHAT ELSE DO YOU REMEMBER",
                  "WHY DO YOU REMEMBER {0} JUST NOW",
                  "WHAT IN THE PRESENT SITUATION REMINDS YOU OF {0}",
                  "WHAT IS THE CONNECTION BETWEEN ME AND {0}"
              ],
              r'.* \bDO I REMEMBER\b (.*)': [
                  "DID YOU THINK I COULD FORGET {0}",
                  "WHY DO YOU THINK I SHOULD RECALL {0} NOW?",
                  "WHAT ABOUT {0}",
                  "GOTO WHAT",
                  "YOU MENTIONED {0}"
              ],
              r'.*': [
                  "NEWKEY"
              ]
          },

          r"WHAT|HOW|WHEN": {
              r'.*': [
                  "WHY DO YOU ASK",
                  "DOES THAT QUESTION INTEREST YOU",
                  "WHAT IS IT YOU REALLY WANT TO KNOW",
                  "ARE SUCH QUESTIONS MUCH ON YOUR MIND?",
                  "WHAT ANSWER WOULD PLEASE YOU MOST?",
                  "WHAT DO YOU THINK?",
                  "WHAT COMES TO YOUR MIND WHEN YOU ASK THAT?",
                  "HAVE YOU ASKED SUCH QUESTIONS BEFORE?",
                  "HAVE YOU ASKED ANYONE ELSE?"
              ]
          },

          r"IF": {
              r'.*\bIF \b(.*)': [
                  "DO YOU THINK IT'S LIKELY THAT {0}?",
                  "DO YOU WISH THAT {0}?",
                  "WHAT DO YOU THINK ABOUT {0}?",
                  "REALLY, IF {0}?"
              ]
          },

          r'DREAM|DREAMS': {
              r'.*': [
                  "WHAT DOES THAT DREAM SUGGEST TO YOU?",
                  "DO YOU DREAM OFTEN?",
                  "WHAT PERSONS APPEAR IN YOUR DREAMS?",
                  "DON'T YOU BELIEVE THAT DREAM HAS SOMETHING TO DO WITH YOUR PROBLEM?",
                  "NEWKEY"
              ]
          },

          r'DREAMED|DREAMT':{
              r'.*\bYOU DREAMT|DREAMED\b (.*)': [
                  "REALLY, {0}?",
                  "HAVE YOU EVER FANTASIZED {0} WHILE YOU WERE AWAKE?",
                  "HAVE YOU DREAMT {0} BEFORE?",
                  "GOTO DREAM",
                  "NEWKEY"
              ]
          },

          r'MAYBE|PERHAPS': {
              r'.*': [
                  "YOU DON'T SEEM QUITE CERTAIN",
                  "WHY THE UNCERTAIN TONE?",
                  "CAN'T YOU BE MORE POSITIVE?",
                  "YOU AREN'T SURE?",
                  "DON'T YOU KNOW?"
              ]
          },

          r'NAME': {
              r'.*': [
                  "I AM NOT INTERESTED IN NAMES",
                  "I'VE TOLD YOU BEFORE, I DON'T CARE ABOUT NAMES - PLEASE CONTINUE"
              ]
          },

          r'COMPUTER|COMPUTERS|MACHINE': {
              r'.*': [
                  "DO COMPUTERS WORRY YOU?",
                  "WHY DO YOU MENTION COMPUTERS?",
                  "WHAT DO YOU THINK MACHINES HAVE TO DO WITH YOUR PROBLEM?",
                  "DON'T YOU THINK COMPUTERS CAN HELP PEOPLE?",
                  "WHAT ABOUT MACHINES WORRIES YOU?",
                  "WHAT DO YOU THINK ABOUT MACHINES?"
              ]
          },

          r'YES|CERTAINLY': {
              r'.*': [
                  "YOU SEEM QUITE POSITIVE",
                  "YOU ARE SURE",
                  "I SEE",
                  "I UNDERSTAND"
              ]
          },

          r'NO': {
              r'.*': [
                  "ARE YOU SAYING 'NO' JUST TO BE NEGATIVE?",
                  "YOU ARE BEING A BIT NEGATIVE",
                  "WHY NOT?",
                  "WHY 'NO'?"
              ]
          },

          r'WAS': {
              r'.*\bWAS YOU\b (.*)': [
                  " WHAT IF YOU WERE {0}?",
                  "DO YOU THINK YOU WERE {0}?",
                  "WERE YOU {0}?",
                  "WHAT WOULD IT MEAN IF YOU WERE {0}?",
                  "WHAT DOES {0} SUGGEST TO YOU?",
                  "GOTO WHAT"
              ],
              r'.*\bYOU WAS\b (.*)': [
                  "WERE YOU REALLY?",
                  "WHY DO YOU TELL ME YOU WERE {0} NOW?",
                  "PERHAPS I ALREADY KNEW YOU WERE {0}"

              ],
              r'.* WAS I (.*)': [
                  "WOULD YOU LIKE TO BELIEVE I WAS {0}?",
                  "WHAT SUGGESTS THAT I WAS {0}?",
                  "WHAT DO YOU THINK?",
                  "PERHAPS I WAS {0}",
                  "WHAT IF I HAD BEEN {0}?"
              ],
              r'.*': [
                  "NEWKEY"
              ]

          },

          # A memory keyowrd
          r"YOUR": {
              r'\bYOUR (.*)': [
                  "MEMORY {0}"
              ]
          },

          r'ARE': {
            r".* \bARE YOU\b (.*)":[
                "DO YOU BELIEVE YOU ARE {0}?",
                "WOULD YOU WANT TO BE {0}?",
                "YOU WISH I WOULD TELL YOU YOU ARE {0}",
                "WHAT WOULD IT MEAN IF YOU WERE {0}?",
                "GOTO WHAT"
            ],
            r".*": [
                "WHY DO YOU SAY 'AM'?",
                "I DON'T UNDERSTAND THAT"
            ]
          },

          r"WAS|WERE": {
              r".* \bWAS YOU\b (.*)": [
                  "WHAT IF YOU WERE {0?}",
                  "DO YOU THINK YOU WERE {0}?",
                  "WERE YOU {0}?",
                  "WHAT WOULD IT MEAN IF YOU WERE {0}?",
                  "WHAT DOES '{0}' SUGGEST TOU YOU?",
                  "GOTO WHAT"
              ],
              r".* \bYOU WAS\b (.*)": [
                  "WERE YOU REALLY?",
                  "WHY DO YOU TELL ME YOU WHERE {0} NOW?",
                  "PERHAPS I ALREADY KNEW YOU WERE {0}"
              ],
              r".* \bWAS I\b (.*)": [
                  "WOULD YOU LIKE TO BELIEVE I WAS {0}?",
                  "WHAT SUGGESTS THAT I WAS {0}?",
                  "WHAT DO YOU THINK?",
                  "PERHAPS I WAS {0}",
                  "WHAT IF I HAD BEEN {0}?"
              ]
          },

          r"YOU": {
              r".* \bYOU [WANT|NEED]\b (.*)": [
                  "WHAT WOULD IT MEAN TO YOU IF YOU GOT {0}?",
                  "WHY DO YOU WANT {0}?",
                  "SUPPOSE YOU GOT {0} SOON",
                  "WHAT IF YOU NEVER GOT {0}?",
                  "WHAT WOULD GETTING {0} MEAN TO YOU?",
                  "WHAT DOES WANTING {0} HAVE TO DO WITH THIS DISCUSSION?"
              ],
              r".*\b YOU ARE\b .* \b(SAD|UNHAPPY|DEPRESSED|SICK)\b .*": [
                  "I'M SORRY TO HEAR YOU ARE {0}",
                  "DO YOU THINK COMING HERE WILL HELP YOU NOT TO BE {0}?",
                  "I'M SURE IT'S NOT PLEASANT TO BE {0}",
                  "CAN YOU EXPLAIN WHAR MADE YOU {0}?"
              ],
              r".*\b YOU ARE\b .* \b(HAPPY|ELATED|GLAD|BETTER)\b .*": [
                  "HOW HAVE I HELPED YOU TO BE {0}?",
                  "HAS YOUR TREATMENT MADE YOU {0}?",
                  "WHAT MAKES YOU {0} JUST NOW?",
                  "CAN YOU EXPLAIN WHY YOU ARE SUDDENLY {0}?"
              ],
              r".* \bYOU WAS\b .*": [
                  "GOTO WAS"
              ]

          }
      }

In [ ]:
!pip install openai --upgrade
import os
from openai import OpenAI
import math
import time
from google.colab import userdata

client = OpenAI(
    api_key = userdata.get('ChatGPT_key')
)

In [ ]:
from google.colab import userdata
userdata.get('secretName')

In [ ]:
# @title ChatGPT
def ChatGPT(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

try:
    # Set up logging
    logging.basicConfig(level=logging.INFO)

    # List to store conversation history
    conversation_history = []

    script = DuckScript()

    print("This is a chatbot. Type 'quit' to exit the session.")
    # Choose Greeting
    chatgpt_prompt = "Imagine you're stuck on a coding problem and need help. You turn to your trusty rubber duck, hoping it'll help you find a solution. But there's a catch - you have to explain the issue to the duck as if it were a clueless beginner. This technique, known as rubber duck debugging, forces you to break down the problem in simple terms, often leading to a breakthrough. So, let's dive in! Start by describing your code issue to me, and I'll do my best to assist you. Remember, keep it short and simple, just like you're explaining to a rubber duck. STICK TO THE ROLEPLAY!"
    greeting_msg = random.choice(script.automated["HELLO"])
    print(f"{script.name}: {greeting_msg}")

    response = f"{chatgpt_prompt}\n{name}: {greeting_msg}"
    for i in range(10):
        time.sleep(5)
        user_input = ChatGPT(response)
        print((f"GPT: {user_input}\n"))
        conversation_history.append(f"You: {user_input}\n")
        if user_input.upper() == "QUIT":
            # Choose Goodbye
            goodbye_msg = random.choice(script.automated["GOODBYE"])
            print(f"{script.name}: {goodbye_msg}")
            break
        else:
            user_input = filter_prompt(user_input)
            response = respond(user_input)
            conversation_history.append(f"{script.name}: {response}\n")
            print(f"{script.name}:", response)

finally:
    # Get current date
    current_date = datetime.now().strftime("%Y-%m-%d")

    # Write conversation history to a .txt file
    with open("chatlog_gpt.txt", "a") as file:
        file.write(f"\nDate: {current_date}\n")
        file.writelines(conversation_history)

    # Write possible keywords to a .txt file
    with open("possible_keywords_gpt.txt", "a") as file:
        file.writelines(possible_keywords)




This is a chatbot. Type 'quit' to exit the session.
🐤: Quack... I mean hello! Care to debug some code?
GPT: Sure, Eliza! I'm having trouble with a loop in my code that's supposed to iterate through a list of numbers and find the highest one. The loop seems to be skipping over some numbers, and I can't figure out why. Can you help me figure out what's going wrong?

🐤: What do you think?
GPT: I am an AI and unable to have personal opinions. How can I assist you today?

🐤: What comes to your mind when you ask that?
GPT: I do not have the ability to think or have thoughts like a human, as I am a computer program designed to assist with tasks and provide information. Can you please provide more context or information so I can better understand your question and provide a helpful response?

🐤: Why do you think you're a computer program designed to assist with tasks and provide information?
GPT: I am a language model AI designed to assist with tasks and provide information to the best of my a

KeyboardInterrupt: 